In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import random_split
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
total_data = torch.utils.data.ConcatDataset([train_data, test_data])



Files already downloaded and verified
Files already downloaded and verified


In [2]:
total_dataset_size = len(total_data)
train_dataset_size = int(total_dataset_size * 0.8)
test_dataset_size = int(total_dataset_size * 0.1)
validate_dataset_size = total_dataset_size - train_dataset_size - test_dataset_size

In [3]:
train_dataset, validate_dataset, test_dataset = random_split(total_data, [train_dataset_size, validate_dataset_size, test_dataset_size])
print(len(train_dataset))
print(len(validate_dataset))
print(len(test_dataset))

48000
6000
6000


In [4]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=10, shuffle=True)
validateloader = torch.utils.data.DataLoader(validate_dataset, batch_size=10, shuffle=False)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=False)

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
     def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,16,3,1, padding=1)
        self.conv2 = nn.Conv2d(16, 32,3,1, padding=1)
        self.conv3 = nn.Conv2d(32, 64,3,1, padding=1)
        
        self.fc1 = nn.Linear( 4*4*64, 500)  
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(500, 10)
     def forward(self, x):
         x = F.relu(self.conv1(x))
         x = F.max_pool2d(x, 2, 2)
         x = F.relu(self.conv2(x))
         x = F.max_pool2d(x, 2, 2)
         x = F.relu(self.conv3(x))
         x = F.max_pool2d(x, 2, 2)
         x = x.view(-1, 4 * 4 * 64)
         x = F.relu(self.fc1(x))
         x = self.dropout1(x)
         x = self.fc2(x)
         return x


model = CNN()
print(model)


CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=1024, out_features=500, bias=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=1024, out_features=500, bias=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [8]:
num_epochs = 30 
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}")

print('Finished Training')


Epoch 10, Loss: 0.3675892642478963
Epoch 20, Loss: 0.23721938621628463
Epoch 30, Loss: 0.21194110318640658
Finished Training


In [9]:
model.eval()  

total_val_loss = 0.0
total_samples = 0


with torch.no_grad():
    for inputs, labels in validateloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        
        # loss
        loss = criterion(outputs, labels)
        
        # total loss
        total_val_loss += loss.item() * inputs.size(0)
        
        
        total_samples += inputs.size(0)

#  average validation loss
average_val_loss = total_val_loss / total_samples

print(f'Validation Loss: {average_val_loss}')


Validation Loss: 1.856115629663885


In [10]:

model.eval()


correct = 0
total = 0
predicted_labels = []
true_labels = []


with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        
        
        _, predicted = torch.max(outputs, 1)
        
        
        total += labels.size(0)
        
        
        correct += (predicted == labels).sum().item()
        
       
        predicted_labels.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# accuracy
accuracy = correct / total

# F-score
from sklearn.metrics import f1_score
f_score = f1_score(true_labels, predicted_labels, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"F-score: {f_score}")


Accuracy: 0.7163333333333334
F-score: 0.7167505605197851
